In [15]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator


DATADIR = r"C:\Users\acer\Downloads\w1867202\SDGP\Dataset"
CATEGORIES = ['Chickenpox','Mild','Monkeypox','Normal','Severe']
            

In [16]:
img_size = (224, 224)  # Specify the desired image size
batch_size = 32  

datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [17]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

y_train = CATEGORIES

# Create a mapping from class labels to integers
class_labels = np.unique(y_train)
label_to_integer = {label: idx for idx, label in enumerate(class_labels)}

# Convert the string labels to integers
y_train_int = [label_to_integer[label] for label in y_train]

# Calculate the class weights
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(class_labels), y= class_labels)
class_weights_dict = dict(zip(np.unique(y_train_int), class_weights))


In [18]:
import tensorflow as tf

# Define custom loss function with class weights
def weighted_categorical_crossentropy(class_weights):
    def loss(y_true, y_pred):
        # Convert class weights to float32
        class_weights_float32 = tf.cast(class_weights, dtype=tf.float32)

        # Calculate the cross-entropy loss for each sample
        ind_loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)

        # Apply class weights to the loss for each sample
        weights = tf.gather(class_weights_float32, tf.argmax(y_true, axis=1))
        weighted_loss = ind_loss * weights

        # Return the average loss over all samples
        return tf.reduce_mean(weighted_loss)

    return loss


In [19]:
trdata = ImageDataGenerator()
# traindata = training_data
train_data = trdata.flow_from_directory(directory="Dataset",target_size=(224,224))

tsdata = ImageDataGenerator()
test_data = tsdata.flow_from_directory(directory="test", target_size=(224,224))


Found 1328 images belonging to 5 classes.
Found 32 images belonging to 5 classes.


In [20]:
img_size = (224, 224)
batch_size = 32

#Data Augmentation Generator
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load the Training and Validation Data
train_data = datagen.flow_from_directory(
    directory="Dataset",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=True,  
    subset='training'  
)

val_data = datagen.flow_from_directory(
    directory="Dataset",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  
    shuffle=True,  
    subset='validation'  
)

# Separate X_val and y_val from val_data generator
X_val, y_val = [], []
for _ in range(len(val_data)):
    x_val_batch, y_val_batch = next(val_data)
    X_val.extend(x_val_batch)
    y_val.extend(y_val_batch)

# Convert the lists to numpy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)




Found 1328 images belonging to 5 classes.
Found 0 images belonging to 5 classes.


In [21]:
#VGG16 model 
import keras,os
from keras.models import Sequential
from keras.layers import Input,Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from glob import glob


In [22]:
# Base model
IMAGE_SIZE=[224,224]
num_classes = 3
vgg=VGG16(input_shape=IMAGE_SIZE+[num_classes],weights='imagenet',include_top=False)

In [23]:
for layer in vgg.layers:
    layer.trainable=False

In [24]:
folders=glob(r"C:\Users\acer\Downloads\w1867202\SDGP\Dataset\*")
print(len(folders))

5


In [25]:
# Add custom classification layers

x=Flatten()(vgg.output)
prediction=Dense(len(folders),activation='softmax')(x)
model=Model(inputs=vgg.input,outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [26]:
#import ADAM optimizer & compile 

from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.001),
              loss=weighted_categorical_crossentropy(class_weights),
              metrics=['accuracy'])

In [27]:
for layer in vgg.layers:
    layer.trainable=False


In [28]:
# Train ML model

trained_model = model.fit(
    x=train_data,                   
    epochs=10,              
    validation_data=val_data,        
    steps_per_epoch=len(train_data), 
    validation_steps=len(val_data)   
)



Epoch 1/10
42/42 [==============================] - 435s 10s/step - loss: 1.0776 - accuracy: 0.6212
Epoch 2/10
42/42 [==============================] - 442s 11s/step - loss: 0.6067 - accuracy: 0.7658
Epoch 3/10
42/42 [==============================] - 431s 10s/step - loss: 0.4886 - accuracy: 0.8283
Epoch 4/10
42/42 [==============================] - 431s 10s/step - loss: 0.4655 - accuracy: 0.8261
Epoch 5/10
42/42 [==============================] - 433s 10s/step - loss: 0.4116 - accuracy: 0.8502
Epoch 6/10
42/42 [==============================] - 430s 10s/step - loss: 0.4136 - accuracy: 0.8517
Epoch 7/10
42/42 [==============================] - 431s 10s/step - loss: 0.2903 - accuracy: 0.9074
Epoch 8/10
42/42 [==============================] - 430s 10s/step - loss: 0.3402 - accuracy: 0.8886
Epoch 9/10
42/42 [==============================] - 433s 10s/step - loss: 0.2654 - accuracy: 0.9142
Epoch 10/10
42/42 [==============================] - 430s 10s/step - loss: 0.2605 - accuracy: 0.9089

In [29]:
# Save the trained model
model.save('saved_model.h5')

In [30]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


CATEGORIES = ['Chickenpox', 'Mild', 'Monkeypox', 'Normal', 'Severe']
IMG_SIZE = 224

# Load and preprocess the image
img = image.load_img(r"C:\Users\acer\Downloads\w1867202\SDGP\test\Monkeypox\Monkeypox3.png", target_size=(IMG_SIZE, IMG_SIZE))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img.astype('float32') / 255.0

# Load the pre-trained model
model = load_model('saved_model.h5')

# Get the prediction
predicted_img = model.predict(img)
index_predicted_img = np.argmax(predicted_img)
print(CATEGORIES[index_predicted_img])

ValueError: Unknown loss function: 'loss'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.